<a href="https://colab.research.google.com/github/Ajaygenuinedoubt/Object_detection_project/blob/main/Object_etection_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import files
uploaded = files.upload()  # Upload a .mp4 or .avi file


Saving Screen Recording 2024-08-16 192759.mp4 to Screen Recording 2024-08-16 192759.mp4


In [21]:
import cv2
import numpy as np

# Update these filenames to match your uploaded files
args = {
    "prototxt": "MobileNetSSD_deploy.prototxt.txt",
    "model": "MobileNetSSD_deploy.caffemodel",
    "confidence": 0.2,
    "video_path": "/content/video.mp4",       # your uploaded video filename
    "output_path": "output.avi"     # output file
}

# Load class labels
CLASSES = ["aeroplane", "background", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

# Load model
print("[INFO] Loading model...")
net = cv2.dnn.readNetFromCaffe(args["prototxt"], args["model"])

# Load video
cap = cv2.VideoCapture(args["video_path"])
if not cap.isOpened():
    print("[ERROR] Cannot open video.")
else:
    print("[INFO] Processing video...")

# Get video writer set up
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(args["output_path"], fourcc, fps, (width, height))

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)),
                                 0.007843, (300, 300), 127.5)

    net.setInput(blob)
    detections = net.forward()

    # Draw detections
    for i in np.arange(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > args["confidence"]:
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            label = f"{CLASSES[idx]}: {confidence*100:.2f}%"
            cv2.rectangle(frame, (startX, startY), (endX, endY), COLORS[idx], 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(frame, label, (startX, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

    # Write frame to output
    out.write(frame)
    frame_count += 1

# Clean up
cap.release()
out.release()
cv2.destroyAllWindows()
print(f"[INFO] Done. Total frames processed: {frame_count}")


[INFO] Loading model...
[INFO] Processing video...
[INFO] Done. Total frames processed: 1611
